In [131]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [187]:
#unifying names 
data=pd.read_csv("cleaned_train.csv")
data = data.rename(columns={"pickup_BoroCode":"pickup_boro_code","pickup_NTACode":"pickup_nta_code","dropoff_BoroCode":"dropoff_boro_code","dropoff_NTACode":"dropoff_nta_code", "pickup_doy":"pickup_day"})
data=data[data["length_time"]<=4236]
data=data.loc[data["length_time"]>=60] 
data=data.loc[data["trip_distance"]>=0.01]
data=data.drop(["weight"],axis=1)

In [3]:
data

,weight,response,length_time,pickup_boro_code,pickup_nta_code,dropoff_boro_code,dropoff_nta_code,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id,passenger_count,trip_distance,fare_amount,pickup_datetime,pickup_week,pickup_wday
0,1,1.75,520,Manhattan,MN15,Manhattan,MN19,-73.983009,40.766602,-73.967972,40.760677,1,1,0.90,7.0,2015-05-11 18:00:00,20,Monday
1,1,2.95,824,Manhattan,MN21,Manhattan,MN24,-73.989510,40.734470,-73.997185,40.717976,1,1,1.40,10.0,2015-05-06 18:00:00,19,Wednesday
2,1,2.46,1016,Manhattan,MN25,Manhattan,MN27,-74.012741,40.702229,-73.998878,40.713711,2,1,1.14,11.5,2015-05-11 07:00:00,20,Monday
3,1,1.66,550,Manhattan,MN99,Manhattan,MN20,-73.970909,40.767231,-73.979507,40.752331,2,1,1.28,7.5,2015-05-13 11:00:00,20,Wednesday
4,1,1.75,501,Manhattan,MN20,Manhattan,MN22,-73.973869,40.747925,-73.986618,40.730328,1,1,1.40,7.5,2015-05-07 20:00:00,19,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341001,1,2.00,740,Manhattan,MN12,Manhattan,MN36,-73.978676,40.783512,-73.938370,40.850533,2,1,5.84,18.5,2015-05-24 20:00:00,21,Sunday
341002,1,1.00,193,Manhattan,MN14,Manhattan,MN12,-73.982048,40.775452,-73.976860,40.783520,2,6,0.70,4.5,2015-05-20 23:00:00,21,Wednesday
341003,1,2.35,764,Manhattan,MN20,Manhattan,MN22,-73.974472,40.747253,-73.983070,40.726955,1,1,1.70,10.0,2015-05-11 19:00:00,20,Monday
341004,1,2.65,955,Manhattan,MN13,Manhattan,MN27,-73.993607,40.745293,-73.987350,40.722256,1,1,2.20,12.0,2015-05-19 21:00:00,21,Tuesday


In [188]:
y=data["response"]
X=data.drop(["response"], axis=1)
X['vendor_id']=X["vendor_id"].map(str)
X['pickup_boro_code']=X["pickup_boro_code"].map(str)
X['dropoff_boro_code']=X["dropoff_boro_code"].map(str)
X_dummies=pd.get_dummies(X)

In [87]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error

def evaluate_estimator(estimator, X, y): 
    # Make predictions
    y_pred = estimator.predict(X)
    
    
    mae = mean_absolute_error(y, y_pred)
    
    # Create a dictionary to hold the performance metrics
    metrics = {
        'MAE': mae
    }
    
    return metrics


In [88]:
import numpy as np
import matplotlib.pyplot as plt

def plot_residuals(model, X, y):
    """
    Plot residuals of a model.
    
    Parameters:
        model: Fitted regression model (e.g., from scikit-learn)
        X: Independent variables (features)
        y: Observed values
        
    Returns:
        None
    """
    # Make predictions using the model
    y_pred = model.predict(X)

    # Compute residuals
    residuals = y - y_pred

    # Plot residuals against predicted values
    plt.scatter(y_pred, residuals)
    plt.xlabel('Predicted values')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')
    plt.axhline(y=0, color='r', linestyle='--')  # Add a horizontal line at y=0
    plt.show()

In [189]:
import sklearn.preprocessing
poly=sklearn.preprocessing.PolynomialFeatures(degree=2, interaction_only=False, include_bias=False, order='C')

In [190]:
X=pd.DataFrame(poly.fit_transform(X.select_dtypes(exclude=["object"])))
scaler=sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=True, clip=False)
scaler.fit(X)
X=pd.DataFrame(scaler.transform(X))

In [191]:
X=X.dropna(axis=1,how="all")

In [192]:
result=pd.merge(X,X_dummies[X_dummies.columns[10:len(X_dummies.columns)]], how="outer",left_index=True, right_index=True)

In [138]:
result.columns = result.columns.astype(str)

In [62]:
import sklearn.decomposition
pca=sklearn.decomposition.PCA(n_components=1000)
pca.fit(result)

PCA(n_components=1000)

In [63]:
import plotly.express as px
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)

In [13]:
import sklearn.decomposition
pca=sklearn.decomposition.PCA(n_components=100)
pca.fit(result)

PCA(n_components=100)

In [141]:
result= np.asarray(result).astype(np.float32)

In [144]:
len(result[0])

1143

In [178]:
import keras
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn
import sklearn.neural_network
# Load the Iris dataset
callback = keras.callbacks.EarlyStopping(monitor='loss',patience=5)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(result, y, test_size=0.1, random_state=42)

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Attention
from keras.layers import BatchNormalization
import tensorflow as tf
import tensorflow.python.keras.backend as K

model = Sequential()
model.add(Dense(1143, input_shape=(1143,)))
model.add(Dense(500))
model.add(Dropout(0.2))
model.add(Dense(100))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dense(1, activation="relu"))                                


# Compile the model
model.compile(loss="mean_absolute_error",optimizer="Adam")

c:\Users\krzyz\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:73: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [179]:
history=model.fit(X_train, y_train, epochs=40, batch_size=10000, validation_split=0.2)

Epoch 1/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 270ms/step - loss: 3.0490 - val_loss: 1.0769
Epoch 2/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 260ms/step - loss: 0.9857 - val_loss: 0.7643
Epoch 3/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 251ms/step - loss: 0.7575 - val_loss: 0.6937
Epoch 4/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 244ms/step - loss: 0.7220 - val_loss: 0.6786
Epoch 5/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - loss: 0.7073 - val_loss: 0.6736
Epoch 6/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 247ms/step - loss: 0.7063 - val_loss: 0.6522
Epoch 7/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 249ms/step - loss: 0.6955 - val_loss: 0.6873
Epoch 8/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 241ms/step - loss: 0.6828 - val_loss: 0.6404
Epoch 9/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 245ms/step - loss: 0.6710 - val_loss: 0.6562
Epoch 10/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 245ms/step - loss: 0.6719 - val_loss: 0.6854
Epoch 11/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 248ms/step - loss: 0.6605 - val_loss: 0.6247
Epoch 12/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 251ms/step

In [180]:
pred=model.predict(X_test)
p=np.zeros(len(pred))
for i in range(len(pred)):
   p[i]=pred[i][0]
import plotly.express as px


px.scatter(x=np.array(y_test),y=p) #ouupredicts small values


1066/1066 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [182]:
p=pd.Series(p)
for n in range(len(p)):
    if p[n]<0:
        p[n]=0
mean_absolute_error(np.array(y_test),np.round(p,10))

0.6013395291242368

In [184]:
p.describe()

count    34101.000000
mean         2.788527
std          2.028911
min          0.000000
25%          1.539019
50%          2.134827
75%          3.184188
max         14.572379
dtype: float64

In [198]:
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,1289,1290,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538
0,-0.779693,-0.388634,0.166757,0.025051,0.237610,-0.666667,-0.964393,-0.938983,0.0,-0.970257,0.779636,-0.779609,0.779706,-0.779676,-0.959013,-0.992964,-0.986357,-0.797589,0.387599,-0.456939,0.329860,-0.319060,0.667162,0.964398,0.938985,-0.007049,0.164826,-0.243985,0.178929,-0.667765,-0.964366,-0.938920,-0.001007,-0.026860,-0.423072,0.667501,0.964403,0.939004,0.004948,0.235331,-0.667873,-0.964379,-0.938874,0.002550,-0.944444,-0.989726,-0.969298,-0.696970,-0.999352,-0.997347,-0.966070,-0.996199,-0.941628,-0.0500,0.607922,0.303015,-0.130019,-0.019532,-0.185263,0.519796,0.751931,0.732119,-0.000000,0.756503,-0.607877,0.607856,-0.607931,0.607908,0.747736,0.774208,0.769056,0.621875,-0.302208,0.356273,-0.257190,0.248769,-0.520182,-0.751935,-0.732120,0.005496,-0.128514,0.190233,-0.139510,0.520652,0.751910,0.732070,0.000785,0.020942,0.329867,-0.520446,-0.751939,-0.732136,-0.003858,-0.183486,0.520736,0.751920,0.732034,-0.001989,0.736377,0.771683,0.755756,0.543423,0.779188,0.777625,0.753239,0.776730,0.734181,0.038985,0.151036,-0.064807,-0.009736,-0.092343,0.259089,0.374796,0.364920,-0.000000,0.377074,-0.302993,0.302982,-0.303020,0.303008,0.372705,0.385899,0.383332,0.309970,-0.150634,0.177582,-0.128195,0.123998,-0.259282,-0.374798,-0.364921,0.002739,-0.064057,0.094821,-0.069538,0.259516,0.374785,0.364896,0.000391,0.010439,0.164420,-0.259413,-0.374799,-0.364929,-0.001923,-0.091458,0.259558,0.374790,0.364878,-0.000991,0.367043,0.384641,0.376702,0.270866,0.388382,0.387603,0.375447,0.387156,0.365948,0.019432,0.027808,0.004177,0.039623,-0.111171,-0.160819,-0.156582,0.000000,-0.161797,0.130010,-0.130005,0.130021,-0.130016,-0.159922,-0.165584,-0.164482,-0.133004,0.064635,-0.076198,0.055007,-0.053206,0.111254,0.160820,0.156582,-0.001175,0.027486,-0.040686,0.029838,-0.111355,-0.160815,-0.156572,-0.000168,-0.004479,-0.070550,0.111311,0.160821,0.1

In [203]:
data=pd.read_csv("cleaned_test.csv")
test=data
test['vendor_id']=test["vendor_id"].map(str)
test['pickup_boro_code']=test["pickup_boro_code"].map(str)
test['dropoff_boro_code']=test["dropoff_boro_code"].map(str)
test_dummies=pd.get_dummies(test)
test=pd.DataFrame(poly.fit_transform(test.select_dtypes(exclude=["object"])))
test.columns = test.columns.astype(str)
test=pd.DataFrame(scaler.transform(test))
import sklearn.preprocessing
test_result=pd.merge(test,test_dummies[test_dummies.columns[10:len(test_dummies.columns)]], how="outer",left_index=True, right_index=True)

c:\Users\krzyz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:432: UserWarning:

X has feature names, but MinMaxScaler was fitted without feature names



In [201]:
=

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
0,-0.779693,-0.388634,0.166757,0.025051,0.237610,-0.666667,-0.964393,-0.938983,0.0,-0.970257,0.779636,-0.779609,0.779706,-0.779676,-0.959013,-0.992964,-0.986357,-0.797589,0.387599,-0.456939,0.329860,-0.319060,0.667162,0.964398,0.938985,-0.007049,0.164826,-0.243985,0.178929,-0.667765,-0.964366,-0.938920,-0.001007,-0.026860,-0.423072,0.667501,0.964403,0.939004,0.004948,0.235331,-0.667873,-0.964379,-0.938874,0.002550,-0.944444,-0.989726,-0.969298,-0.696970,-0.999352,-0.997347,-0.966070,-0.996199,-0.941628,-0.0500
1,-0.634100,-0.424701,-0.031828,-0.084074,0.020483,-0.666667,-0.944389,-0.898305,-0.5,-0.924708,0.634020,-0.634326,0.634012,-0.634537,-0.935052,-0.982627,-0.968375,-0.683440,0.423702,-0.282364,0.427859,-0.153788,0.667133,0.944392,0.898306,0.480783,-0.033812,-0.167587,-0.059556,-0.668027,-0.944391,-0.898335,-0.491175,0.082278,-0.291032,0.667370,0.944382,0.898292,0.484098,0.018069,-0.668221,-0.944427,-0.898332,-0.486544,-0.944444,-0.984018,-0.956140,-0.712121,-0.998432,-0.994062,-0.949686,-0.991664,-0.910906,-0.5375
2,-0.542146,-0.553604,-0.231097,-0.090401,-0.001203,-0.666667,-0.954791,-0.877966,0.0,-0.885323,0.541913,-0.542830,0.542045,-0.542757,-0.919918,-0.982557,-0.954912,-0.582148,0.552758,-0.163357,0.496267,-0.189776,0.667029,0.954779,0.877921,-0.010976,-0.232977,-0.018432,-0.175815,-0.668290,-0.954829,-0.878128,-0.016259,0.088607,-0.274494,0.667362,0.954784,0.877950,0.000914,-0.003618,-0.668256,-0.954827,-0.878027,-0.008501,-0.944444,-0.986986,-0.949561,-0.696970,-0.998960,-0.994442,-0.956926,-0.988797,-0.886329,-0.0500
3,-0.765326,-0.321494,0.170647,-0.018040,0.195170,-0.666667,-0.949190,-0.932203,0.0,-0.966678,0.765312,-0.765237,0.765302,-0.765366,-0.956649,-0.989406,-0.984461,-0.784559,0.320401,-0.420020,0.328313,-0.255646,0.667217,0.949205,0.932224,-0.005451,0.168719,-0.277257,0.154322,-0.667760,-0.949151,-0.932136,-0.000858,0.016232,-0.413196,0.667449,0.949196,0.932212,0.003442,0.192847,-0.667941,-0.949181,-0.932114,0.000586,-0.944444,-0.985388,-0.967105,-0.696970,-0.998689,-0.995939,-0.951591,-0.995554,-0.935484,-0.0500
4,-0.788793,-0.337919,0.051326,-0.044602,0.083290,-0.666667,-0.944389,-0.932203,-0.5,-0.972419,0.788764,-0.788819,0.788741,-0.788951,-0.960511,-0.989445,-0.985898,-0.816723,0.336839,-0.311965,0.355944,-0.169598,0.667203,0.944404,0.932220,0.482746,0.049346,-0.203962,0.022660,-0.667918,-0.944373,-0.932185,-0.488147,0.042796,-0.323463,0.667417,0.944390,0.932202,0.485409,0.080891,-0.668121,-0.944410,-0.932169,-0.483783,-0.944444,-0.984018,-0.967105,-0.712121,-0.998432,-0.995555,-0.949686,-0.995554,-0.940092,-0.5375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341001,-0.674330,-0.364589,0.271272,0.135630,0.694501,-0.666667,-0.766753,-0.783051,0.5,-0.939354,0.674302,-0.674098,0.674524,-0.673561,-0.941673,-0.934859,-0.947070,-0.685959,0.363532,-0.545729,0.236750,-0.688527,0.667182,0.766800,0.783108,-0.495096,0.269432,-0.246789,0.519467,-0.667628,-0.766483,-0.782830,0.486135,-0.137407,-0.787672,0.667634,0.766911,0.783244,-0.473768,0.693251,-0.667141,-0.766152,-0.782362,0.504299,-0.944444,-0.933333,-0.918860,-0.681818,-0.972716,-0.953948,-0.766720,-0.970125,-0.788940,0.4625
341002,-0.936303,-0.383300,0.221455,-0.008151,0.353758,1.000000,-0.972394,-0.972881,0.5,-0.996249,0.936232,-0.936198,0.936244,-0.936188,-0.908725,-0.997984,-0.997191,-0.935433,0.382260,-0.507812,0.351612,-0.413315,-0.997000,0.972399,0.972875,-0.495563,0.219566,-0.309460,0.279994,0.993840,-0.972367,-0.972817,0.484130,0.006342,-0.560680,-0.995233,0.972399,0.972880,-0.479043,0.351645,0.993877,-0.972368,-0.972744,0.487743,1.000000,-0.952055,-0.881579,0.909091,-0.999608,-0.998690,-0.972357,-0.998755,-0.969585,0.4625

In [211]:
test = test_result.reindex(pd.DataFrame(result).columns, axis=1, fill_value=0)

In [212]:
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,pickup_boro_code_Manhattan,pickup_boro_code_Queens,pickup_boro_code_Staten Island,pickup_boro_code_The Bronx,pickup_nta_code_BK09,pickup_nta_code_BK17,pickup_nta_code_BK19,pickup_nta_code_BK28,pickup_nta_code_BK30,pickup_nta_code_BK31,pickup_nta_code_BK32,pickup_nta_code_BK33,pickup_nta_code_BK34,pickup_nta_code_BK35,pickup_nta_code_BK37,pickup_nta_code_BK38,pickup_nta_code_BK40,pickup_nta_code_BK41,pickup_nta_code_BK42,pickup_nta_code_BK43,pickup_nta_code_BK44,pickup_nta_code_BK50,pickup_nta_code_BK58,pickup_nta_code_BK60,pickup_nta_code_BK61,pickup_nta_code_BK63,pickup_nta_code_BK64,pickup_nta_code_BK68,pickup_nta_code_BK69,pickup_nta_code_BK72,pickup_nta_code_BK73,pickup_nta_code_BK75,pickup_nta_code_BK76,pickup_nta_code_BK77,pickup_nta_code_BK78,pickup_nta_code_BK79,pickup_nta_code_BK81,pickup_nta_code_BK82,pickup_nta_code_BK83,pickup_nta_code_BK85,pickup_nta_code_BK88,pickup_nta_code_BK90,pickup_nta_code_BK91,pickup_nta_code_BK95,pickup_nta_code_BK99,pickup_nta_code_BX06,pickup_nta_code_BX09,pickup_nta_code_BX10,pickup_nta_code_BX13,pickup_nta_code_BX14,pickup_nta_code_BX22,pickup_nta_code_BX26,pickup_nta_code_BX29,pickup_nta_code_BX30,pickup_nta_code_BX34,pickup_nta_code_BX35,pickup_nta_code_BX36,pickup_nta_code_BX39,pickup_nta_code_BX40,pickup_nta_code_BX41,pickup_nta_code_BX43,pickup_nta_code_BX46,pickup_nta_code_BX52,pickup_nta_code_BX59,pickup_nta_code_BX63,pickup_nta_code_BX75,pickup_nta_code_MN01,pickup_nta_code_MN03,pickup_nta_code_MN04,pickup_nta_code_MN06,pickup_nta_code_MN09,pickup_nta_code_MN11,pickup_nta_code_MN12,pickup_nta_code_MN13,pickup_nta_code_MN14,pickup_nta_code_MN15,pickup_nta_code_MN17,pickup_nta_code_MN19,pickup_nta_code_MN20,pickup_nta_code_MN21,pickup_nta_code_MN22,pickup_nta_code_MN23,pickup_nta_code_MN24,pickup_nta_code_MN25,pickup_nta_code_MN27,pickup_nta_code_MN28,pickup_nta_code_MN31,pickup_nta_code_MN32,pickup_nta_code_MN33,pickup_nta_code_MN34,pickup_nta_code_MN35,pickup_nta_code_MN36,pickup_nta_code_MN40,pickup_nta_code_MN50,pickup_nta_code_MN99,pickup_nta_code_QN01,pickup_nta_code_QN02,pickup_nta_code_QN03,pickup_nta_code_QN06,pickup_nta_code_QN07,pickup_nta_code_QN17,pickup_nta_code_QN18,pickup_nta_code_QN19,pickup_nta_code_QN20,pickup_nta_code_QN21,pickup_nta_code_QN22,pickup_nta_code_QN25,pickup_nta_code_QN26,pickup_nta_code_QN27,pickup_nta_code_QN28,pickup_nta_code_QN29,pickup_nta_code_QN30,pickup_nta_code_QN31,pickup_nta_code_QN35,pickup_nta_code_QN37,pickup_nta_code_QN38,pickup_nta_code_QN42,pickup_nta_code_QN45,pickup_nta_code_QN47,pickup_nta_code_QN48,pickup_nta_code_QN49,pickup_nta_code_QN50,pickup_nta_code_QN52,pickup_nta_code_QN53,pickup_nta_code_QN54,pickup_nta_code_QN55,pickup_nta_code_QN56,pickup_nta_code_QN57,pickup_nta_code_QN60,pickup_nta_code_QN61,pickup_nta_code_QN63,pickup_nta_code_QN68,pickup_nta_code_QN70,pickup_nta_code_QN71,pickup_nta_code_QN72,pickup_nta_code_QN76,pickup_nta_code_QN98,pickup_nta_code_QN99,pickup_nta_code_SI22,pickup_nta_code_SI35,pickup_nta_code_SI37,dropoff_boro_code_Brooklyn,dropoff_boro_code_Manhattan,dropoff_boro_code_Queens,dropoff_boro_code_Staten Island,dropoff_boro_code_The Bronx,dropoff_nta_code_BK09,dropoff_nta_code_BK17,dropoff_nta_code_BK19,dropoff_nta_code_BK21,dropoff_nta_code_BK23,dropoff_nta_code_BK25,dropoff_nta_code_BK26,dropoff_nta_code_BK27,dropoff_nta_code_BK28,dropoff_nta_code_BK29,dropoff_nta_code_BK30,dropoff_nta_code_BK31,dropoff_nta_code_BK32,dropoff_nta_code_BK33,dropoff_nta_code_BK34,dropoff_nta_code_BK35,dropoff_nta_code_BK37,dropoff_nta_code_BK38,dropoff_nta_code_BK40,dropoff_nta_code_BK41,dropoff_nta_code_BK42,dropoff_nta_code_BK43,dropoff_nta_code_BK44,dropoff_nta_code_BK45,dropoff_nta_code_BK46,dropoff_nta_code_BK50,dropoff_nta_code_BK58,dropoff_nta_code_BK60,dropoff_nta_code_BK61,dropoff_nta_code_BK63,dropoff_nta_code_BK64,dropoff_nta_code_BK68,dropoff_nta_code_

In [1]:
pred=model.predict(test)
p=np.zeros(len(pred))
for i in range(len(pred)):
   p[i]=pred[i][0]

NameError: name 'model' is not defined

In [128]:
pd.DataFrame(p).to_csv("result_NN.txt",sep="\t", index=False,header=False)

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores=learning_curve(estimator=rf_regressor,X=x_train,y=y_train)
plt.plot(train_sizes,train_scores),plt.plot(train_sizes,valid_scores)
